At the heart of many tensor network algorithms are tensor decomposition and compression operations. These are particularly useful for Matrix Product State (MPS) based algorithms where they are used to compress virtual bonds between tensors. This can lead to more memory efficient representations and also more efficient compute operations.

## Tensor Decomposition

In PicoQuant, we have implemented a tensor decomposition operation which works by:

1. Reshaping the tensor to matrix
2. Applying matrix decomposition methods (SVD used at present but QR should work also)
3. Applying a cutoff and discarding any singular values and corresponding matrix rows/columns below the given threshold
4. Reshaping the resulting matrices to end up with two tensors connected by a virtual bond

We demonstrate its use by decomposing a two-qubit CNOT gate into two tensors, each acting on a different qubit.

In [ ]:
using PicoQuant

The first thing we do is create a tensor network circuit with two qubits with a single CNOT gate acting on these.

In [ ]:
# create empty tensor network circuit with 2 qubits
tn = TensorNetworkCircuit(2)

# add a 2 qubit CNOT gate
println("CX dims: $(size(gate_tensor(:CX)))")
add_gate!(tn, gate_tensor(:CX), [1, 2]);

We add input and output nodes so that when we plot the network, it will display the outgoing index labels.

In [ ]:
add_input!(tn, "00")
add_output!(tn, "00")
plot(tn, showlabels=true)

Next we call the `decompose_tensor!` method to decompose the two qubit gate. We pass the tensor network circuit, the symbol for the node we wish to decompose and two arrays of symbols for the sets of indices each of the decomposed tensors will have.

In [ ]:
new_nodes = decompose_tensor!(tn, :node_1, [:index_1, :index_3], [:index_2, :index_4], threshold=0.2)

This method returns the symbols of the decomposed tensors (also possible to provide these as optional arguments to reuse the same label). Next we check the dimensions of the new nodes and plot the new tensor network.

In [ ]:
getnode(tn, new_nodes[1]).dims, getnode(tn, new_nodes[2]).dims

In [ ]:
plot(tn, showlabels=true)

We see that the rank four tensor has been replaced by two rank three tensors connected with a virtual bond of dimension 2. Because it is often the case that one would want to decompose the tensor for two-qubit gates into tensors acting on each individual gate, this decomposition can be done in PicoQuant when adding the gate to the circuit (`add_gate!` method) or creating a tensor network circuit object from a qiskit circuit object (`convert_qiskit_circ_to_network` method) by passing `decompose=true` to these methods. For example, for the circuit above with the single CNOT gate, this would proceed something like. 

In [ ]:
tn = TensorNetworkCircuit(2)
add_input!(tn, "00")
add_output!(tn, "00")
add_gate!(tn, gate_tensor(:CX), [1, 2], decompose=true)
plot(tn, showlabels=true)

### Manual decomposition

To show a little more clearly what is going on here, we show each step that explicitly when decomposing the two qubit CX gate, involving:
1. Permuting the indices
2. Reshape to a matrix
3. Applying the SVD
4. Applying threshold on singular values and reshaping

In [ ]:
using LinearAlgebra

A = gate_tensor(:CX)

A = permutedims(A, (1, 3, 2, 4)) # 1. permute indices

A = reshape(A, (4, 4)) # 2. reshape to matrix

F = svd(A) # 3. apply SVD

println("Singular values: $(F.S)")

# 4. Apply threshold and reshape
threshold = 0.2
chi = sum(F.S ./ sqrt(sum(F.S.^2)) .> threshold) # apply threshold to normalised singular values
println("$(chi) values over the threshold")

S = F.S[1:chi] # truncate to values over the threshold

# Scale by the square root of singular values and reshape to get left and right nodes
left_node = reshape(F.U[:, 1:chi] * Diagonal(sqrt.(S)), (2, 2, chi))
right_node = reshape(Diagonal(sqrt.(S)) * F.Vt[1:chi, :], (chi, 2, 2));

## Tensor Network Compression

Compression of a tensor network can be achieved via a sequence of contraction and decomposition operations. For states which are not maximally entangled this process can be used for lossless compression. Where states are entanglement, this process can be used to find an approximation with a fidelity related to the bond dimension retained at each decomposition step. Note that tensor network representations of a given state are not unique and the compression procedure is not guaranteed to find the optimal compression except in the special case of 1D systems when the decomposition is applied to bonds in a mixed canonical Matrix Product State (MPS). To understand why this is see [arxiv:1008.3477](https://arxiv.org/abs/1008.3477). 

### Bond and chain compression

In PicoQuant, the `compress_bond!` method compresses a bond between tensors by:

1. Contracting the two tensors to a single tensor
2. Decomposing the tensor back to two separate tensors using the `decompose_tensor!` method explained above

The `compress_tensor_chain!` applies the `compress_bond!` method to the bonds between the given list of tensors, performing two sweeps. This compresses a chain of tensors also called an Matrix Product State (MPS). Next we show an example of this in use.

In [ ]:
# create the tensor network circuit and add 0's for input
tn = TensorNetworkCircuit(3)
add_input!(tn, "000")

Next we look at the nodes and print the size of the tensor for each.

In [ ]:
for node_sym in [:node_1, :node_2, :node_3]
    println("$node_sym dim: $(getnode(tn, node_sym).dims)")
end

We see that each node is a rank one tensor (a vector) of dimension 2 and there are no virtual bonds as this is a product state. We now apply compression along the tensor chain.

In [ ]:
# apply compression
compress_tensor_chain!(tn, [:node_1, :node_2, :node_3])

And print the dimension of the resulting tensors.

In [ ]:
for node_sym in [:node_1, :node_2, :node_3]
    println("$node_sym dim: $(getnode(tn, node_sym).dims)")
end

We see that there are no additional virtual bonds of dimension 1 connecting tensors. It is always possible to add and remove bonds of dimension one.

For a less trivial example we can create a chain with a large bond dimension and then apply the compression. First we create a circuit with a number of random 2 qubit gates applied.

In [ ]:
# create the tensor network circuit and add 0's for input
n = 4
tn = TensorNetworkCircuit(n)
# tn = TensorNetworkCircuit(3, DSLBackend("contract_network.tl", "data.h5", "output.h5", true))
for i in 1:4
    for j in 1:n-1
        add_gate!(tn, randn(2, 2, 2, 2), [j , j+1], decompose=true) # add random 2 qubit gates
    end
end
add_input!(tn, "0"^n)
plot(tn, showlabels=true)

Contracting this network in order will result in the bond dimension between tensors growing.

In [ ]:
inorder_contraction!(tn)

In [ ]:
node_syms = [tn.edges[x].src for x in tn.output_qubits]
for node_sym in node_syms
    println("$node_sym dim: $(getnode(tn, node_sym).dims)")
end

We see that the virtual bond between tensors has grown sigificantly (note that we have to do a little work to get the nodes in the correct ordering by looking at the source for each of the output edges). We next compress a single bond with the `compress_bond!` method before compressing the rest of the chain with the `compress_tensor_cahin!` method.

In [ ]:
compress_bond!(tn, :node_47, :node_49)

In [ ]:
node_syms = [tn.edges[x].src for x in tn.output_qubits]
for node_sym in node_syms
    println("$node_sym dim: $(getnode(tn, node_sym).dims)")
end

We see that the bond dimension between the first two tensors has been reduced from 256 to 2. We can compress the rest of the chain with.

In [ ]:
compress_tensor_chain!(tn, node_syms)

In [ ]:
for node_sym in node_syms
    println("$node_sym dim: $(getnode(tn, node_sym).dims)")
end

Which results in the fully compressed state. Note that the bond dimension in each case is less than the maximum bound, which for an open MPS is $min(d_A, d_B)$ where $d_A$ and $d_B$ are the dimensions of the Hilbert space of the subsystems on either side of the bond.

## MPS contraction

Using these compression operations, we can contract a tensor network while keeping the bond dimension from 

1. Growing beyond the maximum bound in the case where exact contraction is of interest
2. Growing beyond a given bound where approximate results are acceptable

The `contract_mps_tensor_network_circuit!` function does exactly this which we will now demonstrate with some examples.

### Simple Preparation Circuit

We demonstrate contraction with the MPS contraction approach using a simple random preparation circuit which contains alternating layers of single qubit gates and entangling gates. First we create the circuit

In [ ]:
number_qubits = 6
depth = 10
circ = create_simple_preparation_circuit(number_qubits, depth)
circ.draw()

We contract using the full wavefunction approach to get the expected output state vector.

In [ ]:
tn = convert_qiskit_circ_to_network(circ)
add_input!(tn, "0"^number_qubits)
output_node = full_wavefunction_contraction!(tn, "vector")
psi_exact = load_tensor_data(tn, output_node);

Next we contract using the MPS contraction approach with no cut off on the bond dimension which should provide an exact state. We also print the dimensions of the final tensors to show the how large the bond dimension grows.

In [ ]:
tn = convert_qiskit_circ_to_network(circ, decompose=true, transpile=true)
add_input!(tn, "0"^number_qubits)
mps_nodes = contract_mps_tensor_network_circuit!(tn)
@show [tn.nodes[x].dims for x in mps_nodes]
psi_exact_mps = calculate_mps_amplitudes!(tn, mps_nodes);

Next we contract with different bond dimension cutoffs and plot the probability amplitudes of the output state along side the exact results.

In [ ]:
import Plots

Plots.plot(abs.(psi_exact).^2, label="Full WF Exact")
p = Plots.plot!(abs.(psi_exact_mps).^2, label="MPS Exact")
println("MPS exact has overlap: $(abs(psi_exact' * psi_exact_mps)^2)")
for max_rank in [2, 4, 6]
    tn = convert_qiskit_circ_to_network(circ, decompose=true, transpile=true)
    add_input!(tn, "0"^number_qubits)
    mps_nodes = contract_mps_tensor_network_circuit!(tn, max_rank=max_rank)
    psi_approx_mps = calculate_mps_amplitudes!(tn, mps_nodes)
    println("Max rank $(max_rank) has overlap: $(abs(psi_exact' * psi_approx_mps)^2)")
    global p = Plots.plot!(abs.(psi_approx_mps).^2, label="MPS Bond $(max_rank)", ylabel="Probability amplitudes")
end
p

We can see that as the bond dimension cut off is increated, the quality of the approximation to the exact state and overlap increases. Using this approach it's possible to get good approximations of the output state more efficiently than full wave-function simulation.

### Access output amplitudes

In the above calculations, following the MPS contraction we used the `calculate_mps_amplitudes!` function. This contracts the resulting MPS tensors into a single tensor and then reshapes this into a vector of output amplitudes. For large systems this will not be feasible due to the memory required to store all the amplitudes. Instead it is possible to access individual amplitudes of the resulting MPS state which can be done using the MPSState type. This structure provides an array like interface to the amplitudes which accepts either a configuration string or index.

We demonstrate this by contracting a 1000 qubit GHZ state preparation circuit and returning only a subset of the amplitudes. This would not be possible using the full wave-function simulation approach.

In [ ]:
n = 1000
circ = create_ghz_preparation_circuit(n)
tn = convert_qiskit_circ_to_network(circ, decompose=true)
add_input!(tn, "0"^n)
mps_nodes = contract_mps_tensor_network_circuit!(tn)
psi = MPSState(tn, mps_nodes);

In [ ]:
@show psi["0"^n]
@show psi["1"^n]
@show psi[join(rand(["0", "1"], n))]